In [1]:
!ls templates/

404.html          doc.html          index.html        onelist.html
comparelists.html hg38_gallery.html mm10_gallery.html stat.html


In [2]:
import re

# Replacing all hard-coded urls with relative links

In [150]:
genelist_str = '<td><a href="http://lisa.cistrome.org/gallery/13_down.txt">down</a><span> & </span><a href=http://lisa.cistrome.org/gallery/13_up.txt>up</a></td>'
gallery_str = '<td><div class="col"><a class="" href="http://lisa.cistrome.org/new_gallery/20_combined.html">Combined</a></div></td>'

In [151]:
species = 'mm10'

In [170]:
GENELIST_PARSER = re.compile('"*http://lisa.cistrome.org/gallery/(\d+_(up|down).txt)"*')

def replace_genelists(html_line, species):
    return re.sub(GENELIST_PARSER, 
        "\"{{ url_for('get_gallery_genelist', species='" + species + "', filename='\\1') }}\"", 
        html_line)

print(replace_genelists(test_str, 'mm10'))

<td><a href="{{ url_for('get_gallery_genelist', species='mm10', filename='13_down.txt') }}">down</a><span> & </span><a href="{{ url_for('get_gallery_genelist', species='mm10', filename='13_up.txt') }}">up</a></td>


In [167]:
GALLERY_PARSER = re.compile('"*http://lisa.cistrome.org/new_gallery[_m]*/(\d+_combined.html)"*')

def replace_examples(html_line, species):
    return re.sub(GALLERY_PARSER, 
        "\"{{ url_for('get_gallery_example', species='" + species + "', filename ='\\1') }}\"",
        html_line)

print(replace_examples(gallery_str, 'mm10'))

<td><div class="col"><a class="" href="{{ url_for('get_gallery_example', species='mm10', filename ='20_combined.html')}}">Combined</a></div></td>


In [171]:
print(replace_examples(replace_genelists(gallery_str, 'mm10'), 'mm10'))

<td><div class="col"><a class="" href="{{ url_for('get_gallery_example', species='mm10', filename ='20_combined.html')}}">Combined</a></div></td>


In [172]:
with open('templates/new_gallery_mm.html', 'r') as html:
    with open('templates/fixed_gallery.html', 'w') as fixed_html:
        for line in html:
            print(
                replace_examples(replace_genelists(line, 'mm10'), 'mm10'),
                file = fixed_html
            )

# Extracting hg38 and mm10 genelists

In [17]:
genelist_match = re.compile(r'filename *= *\'(\d+_(up|down).txt)\'')

In [18]:
test_str = "<td><a href=\"{{ url_for('get_gallery_genelist', species='hg38', filename='4_down.txt') }}\">down</a><span> & </span><a href=\"{{ url_for('get_gallery_genelist', species='hg38', filename='4_up.txt') }}\">up</a></td>"

In [27]:
with open('templates/mm10_gallery.html', 'r') as html:    
    for line in html:
        search = re.findall(genelist_match, line)
        if not search is None:
            for genelist in search:
                print(genelist[0])

13_down.txt
13_up.txt
20_down.txt
20_up.txt
37_down.txt
37_up.txt
58_down.txt
58_up.txt
59_down.txt
59_up.txt
60_down.txt
60_up.txt
66_down.txt
66_up.txt
89_down.txt
89_up.txt
90_down.txt
90_up.txt
135_down.txt
135_up.txt
152_down.txt
152_up.txt
154_down.txt
154_up.txt
155_down.txt
155_up.txt
156_down.txt
156_up.txt
157_down.txt
157_up.txt
158_down.txt
158_up.txt
159_down.txt
159_up.txt
160_down.txt
160_up.txt
161_down.txt
161_up.txt
162_down.txt
162_up.txt
163_down.txt
163_up.txt
172_down.txt
172_up.txt
173_down.txt
173_up.txt
174_down.txt
174_up.txt
175_down.txt
175_up.txt
176_down.txt
176_up.txt
177_down.txt
177_up.txt
178_down.txt
178_up.txt
180_down.txt
180_up.txt
181_down.txt
181_up.txt
182_down.txt
182_up.txt
183_down.txt
183_up.txt
187_down.txt
187_up.txt
189_down.txt
189_up.txt
199_down.txt
199_up.txt
220_down.txt
220_up.txt
222_down.txt
222_up.txt
234_down.txt
234_up.txt
235_down.txt
235_up.txt
236_down.txt
236_up.txt
237_down.txt
237_up.txt
238_down.txt
238_up.txt
239_down.t

In [25]:
re.findall(genelist_match, test_str)

[('4_down.txt', 'down'), ('4_up.txt', 'up')]

# Parsing metadata

In [4]:
import bs4
import re
import json
import os
import glob
import shutil

In [22]:
def extract_gallery_metadata(species):
    
    with open('templates/{}_gallery.html'.format(species), 'r') as f:    
        soup = bs4.BeautifulSoup(f.read(), 'html.parser')

    headers = [x.get_text() for x in soup.find_all('th')]

    def get_id(link_text):
        search = re.search("filename *= *'(\d+)_(up|down).txt'", link_text)
        if not search is None:
            return search.groups()[0]


    title_map = dict(
        cell_type = 'Cell Type',
        geo_id = 'GEO ID',
        hs_gene_symbol = 'Factor Perturbed',
        organism = 'Organism',
        pert_type = 'Perturbation Type',
    )

    all_metadata = []
    for row in soup.find_all('tr')[1:]:
        metadata = dict()
        for data, header in zip(row.find_all('td'), headers):
            if header == 'gene_set_type':
                metadata['id'] = dict(text = get_id(data.find_all('a')[0].get('href')), link = None)
            elif not header == 'Combined':
                try:
                    link = data.find_all('a')[0].get('href')
                except IndexError:
                    link = None
                metadata[title_map[header]] = dict(text = data.get_text(), link = link)

        all_metadata.append(metadata)


    with open(os.path.join('..','metadata', species + '_gallery_metadata.json'),'w') as f:
        print(json.dumps(all_metadata), file = f)
        
    return all_metadata
        
hg38_metadata = extract_gallery_metadata('hg38')
mm10_metadata = extract_gallery_metadata('mm10')

In [42]:
def rename_gallery_files(species, metadata):
    
    for example in metadata:
        
        task_id = species + '_gallery_example_' + example['id']['text']
        task_dir = os.path.join('..','results',task_id)
        
        if os.path.isdir(task_dir):
            
            try:
            
                for direction, new_prefix in zip(['up','down'],
                        [example['Factor Perturbed']['text'] + '_Up-regulated', example['Factor Perturbed']['text'] + '_Down-regulated']):

                    prefix = example['id']['text'] + '_' + direction + '.txt'

                    for suffix in ['.lisa.tsv','.metadata.json']:

                        _from = os.path.join(task_dir, prefix + suffix)
                        _to = os.path.join(task_dir, new_prefix + suffix)                        

                        if not os.path.isfile(_to):

                            os.rename(
                                os.path.join(task_dir, prefix + suffix), 
                                os.path.join(task_dir, new_prefix + suffix)
                            )


                    _to = os.path.join(task_dir, new_prefix)
                    if not os.path.isfile(_to):

                        os.rename(os.path.join('..',species + '_genelists', prefix), 
                              _to)
                        
            except FileNotFoundError:
                
                print(task_id)
                

rename_gallery_files('mm10', mm10_metadata)

mm10_gallery_example_292
